# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [80]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using ManifoldDiff
using FiniteDiff
using FiniteDifferences

In [84]:
include("../../../src/decompositions/naive_SVD.jl")
include("../../../src/utils/curvature_corrected_loss.jl")
include("../../../src/decompositions/curvature_corrected_low_rank_approximation.jl")

curvature_reweighed_low_rank_approximation (generic function with 1 method)

### Load data and construct manifold ###

In [49]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 5  # 100


5

In [50]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[1]

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


In [23]:
# Export slice image
asymptote_export_SPD("results/artificial1D_orig.asy", data=data);

### Construct low rank approximation ###

In [51]:
q = mean(M, data)
# q = 1. * Matrix(I, 3, 3)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [52]:
# TOY -- test
# compute curvature corrected loss
Ξ = 1/2 .* log_q_data
loss = curvature_corrected_loss(M, e, data, Ξ, 1)

0.0731933024500142

In [85]:
# TOY -- test
curvature_reweighed_low_rank_approximation(M, e, data, 1);

[[0.0032603164109766825 -0.011612164491239595 -0.006220519029908633; -0.011612164491239594 -0.01238433336169978 0.06425941685921388; -0.006220519029908634 0.0642594168592139 0.05404868213545831], [3.941273828877989e-5 0.023489998075394952 -0.0010714421026251602; 0.023489998075394952 0.03414750422585027 -0.023176174824101968; -0.0010714421026251602 -0.02317617482410196 0.057112216206355024], [0.0011130689969645058 -0.05154273519884497 -0.03537476423598776; -0.05154273519884497 0.018759775700455047 0.017287146648717097; -0.03537476423598776 0.017287146648717097 0.048777688135281924], [0.0018188140743078396 0.06672709416577406 0.07412698960696018; 0.06672709416577408 0.025567210381737693 0.02067787984845943; 0.07412698960696018 0.020677879848459424 0.06802077552367838], [-0.0016162933613197983 0.08840863625067531 -0.04969025275349079; 0.08840863625067531 -0.04326390060280254 -0.05459751016594417; -0.04969025275349079 -0.05459751016594417 -0.0472170937581457]]


LoadError: MethodError: no method matching getindex(::ProductRepr{Tuple{Matrix{Float64}, Vector{Float64}, Matrix{Float64}}}, ::Int64)
[0mClosest candidates are:
[0m  getindex(::Union{ProductRepr, ArrayPartition}, [91m::SemidirectProductGroup[39m, [91m::Union{Colon, Integer, Val, AbstractVector}[39m) at ~/.julia/packages/Manifolds/1oKAJ/src/groups/semidirect_product_group.jl:260
[0m  getindex(::Union{ProductRepr, ArrayPartition}, [91m::ProductGroup[39m, [91m::Union{Colon, Integer, Val, AbstractVector}[39m) at ~/.julia/packages/Manifolds/1oKAJ/src/groups/product_group.jl:340
[0m  getindex(::ProductRepr, [91m::ProductManifold[39m, [91m::Union{Colon, Integer, Val, AbstractVector}[39m) at ~/.julia/packages/Manifolds/1oKAJ/src/manifolds/ProductManifold.jl:639
[0m  ...

In [23]:
N_R_q, N_U = naive_SVD(M,q, data);
CR_R_q, CR_U = curvature_reweighed_SVD(M,q, data);

In [24]:
ref_distance = sqrt(sum(distance.(Ref(M), Ref(q), data).^2))

N_tangent_distances_r = zeros(d)
N_distances_r = zeros(d)
CR_tangent_distances_r = zeros(d)
CR_distances_r = zeros(d)

for rank in 1:d
    N_log_q_data_r = Symmetric.([sum([N_R_q[i] * N_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    CR_log_q_data_r = Symmetric.([sum([CR_R_q[i] * CR_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    # expoentiate back
    N_data_r = exp.(Ref(M), Ref(q), N_log_q_data_r)
    CR_data_r = exp.(Ref(M), Ref(q), CR_log_q_data_r)


    # compute relative tangent space error
    N_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - N_log_q_data_r).^2)) / ref_distance
    CR_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - CR_log_q_data_r).^2)) / ref_distance


    # compute relative manifold error
    N_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, N_data_r).^2)) / ref_distance
    CR_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, CR_data_r).^2)) / ref_distance
end

In [25]:
plot(1:d, [N_tangent_distances_r, CR_tangent_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/artificial1D_tangent_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_tangent_rank.png"

In [26]:
plot(1:d, [N_distances_r, CR_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/artificial1D_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_rank.png"